In [26]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [37]:
# Replace with the actual URL of the webpage that lists the smartphones
urls = ["https://www.verizon.com/smartphones/","https://www.verizon.com/smartphones/page-2/", "https://www.verizon.com/smartphones/page-3/", "https://www.verizon.com/smartphones/page-4/"]

smartphone_links = set()
for url in urls:
    
    # Send a GET request to the server and get the content of the webpage
    response = requests.get(url)
    # If request was successful (HTTP Status Code 200)
    if response.status_code == 200:
        # Parse the page content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find links to all smartphones - Replace with actual HTML tag and class/ID
        grid = soup.find("div", id="col")
        smartphone_links_a = grid.find_all("a", href=True)

        # Loop through each link and store in set
        for link in smartphone_links_a:
            if link['href'][0:8] == "https://":
                smartphone_links.add(link['href'])
phones = []
driver = webdriver.Chrome()
smartphone_links = sorted(list(smartphone_links))
for spec_url in smartphone_links:
    
    print(spec_url)
    
    # Navigate to the webpage
    driver.get(spec_url)
    try:
        if driver.find_element(By.TAG_NAME, "h1").text == "something went wrong, please try after some time":
            continue
    except:
        pass
    driver.execute_script("window.  scrollTo(0, document.body.scrollHeight / 2);")
    # Wait for the page to load completely
    time.sleep(2)

    curr_phone = {}
    curr_phone["name"] = driver.find_element(By.TAG_NAME, "h1").find_element(By.TAG_NAME, "span").text
    html = driver.page_source
    for line in html.split('\n'):
        if line.startswith("window.APP_STATE"):
            script = line
            break
    index = 0
    colors = []
    images = []
    for i in range(3):
        index = script.find('"label"', index)
        colors.append(script[index+8:script.find('"', index+10)])
        index = script.find('"images"', index)
        images.append(script[index+11:script.find('"', index+11)])
    
    curr_phone["colors"] = colors
    curr_phone["images"] = images
    curr_phone["url"] = spec_url

    # Find the 'Specs' button and click it - Replace with actual XPATH or CSS Selector
    wait = WebDriverWait(driver, 10)
    specs_button = wait.until(EC.visibility_of_element_located((By.ID, 'FEATURESTAB')))
    retries = 100
    for i in range(retries):
        try:
            specs_button.click()
            break
        except:
            time.sleep(1)
            continue

    # Wait for the specs to load
    time.sleep(1)

    specs = driver.find_elements(By.CLASS_NAME, "pb-8")
    for spec in specs:
        sections = spec.find_elements(By.CLASS_NAME, "py-8")
        for section in sections:
            subsections = section.find_elements(By.TAG_NAME, "p")
            title = subsections[0].text
            description = subsections[1].text
            if title == "Width":
                curr_phone["width"] = description
            elif title == "Depth":
                curr_phone["depth"] = description
            elif title == "Weight":
                curr_phone["weight"] = description
            elif title == "Height":
                curr_phone["height"] = description
            elif title == "Usage Time":
                curr_phone["battery"] = description
            elif title == "Operating System":
                curr_phone["operating_system"] = description
            elif "camera" in title.lower():
                curr_phone["camera"] = description
            elif "storage" in title.lower():
                curr_phone["storage"] = description
    
    phones.append(curr_phone)
    
    # Save curr_phone to json file using pandas
    df = pd.DataFrame(phones)
    df.to_json(f"phones_{len(phones)}.json", orient="records")

https://www.verizon.com/smartphones/apple-iphone-13-mini-certified-pre-owned/
https://www.verizon.com/smartphones/apple-iphone-11-pro-certified-pre-owned/
https://www.verizon.com/smartphones/apple-iphone-11-pro-max-certified-pre-owned/
https://www.verizon.com/smartphones/apple-iphone-12-certified-pre-owned/
https://www.verizon.com/smartphones/apple-iphone-12-mini-certified-pre-owned/
https://www.verizon.com/smartphones/apple-iphone-12-pro-certified-pre-owned/
https://www.verizon.com/smartphones/apple-iphone-12-pro-max-certified-pre-owned/
https://www.verizon.com/smartphones/apple-iphone-12/
https://www.verizon.com/smartphones/apple-iphone-13-certified-pre-owned/
https://www.verizon.com/smartphones/apple-iphone-13-mini-certified-pre-owned/
https://www.verizon.com/smartphones/apple-iphone-13-mini/
https://www.verizon.com/smartphones/apple-iphone-13-pro-certified-pre-owned/
https://www.verizon.com/smartphones/apple-iphone-13-pro-max-certified-pre-owned/
https://www.verizon.com/smartphones

In [ ]:
driver.quit()